# Preparation

all general functions can be declared here

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Clone the GitHub repository
!git clone https://github.com/umd-huang-lab/Mementos.git

Cloning into 'Mementos'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 56 (delta 23), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 144.09 KiB | 3.60 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
import os

In [ ]:
# MEMENTOS_DATA_DIR = "" # replace it with your path
MEMENTOS_DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Grounded Language Processing [146078]/Projects/DATA/"

## Define Our Dataset

We want to make sure that the top 100 of sequential and combined dataset has the same name. If it has the same names, it means that it depicts the same scene and is consistent

In [ ]:
N_SAMPLED = 100
N_INCORRECT = 30

# set the folder of combined & sequential daily life dataset
ALL_COMB_DAILYLIFE_DIR = os.path.join(MEMENTOS_DATA_DIR, "image_dailylife/image")
ALL_SEQ_DAILYLIFE_DIR = os.path.join(MEMENTOS_DATA_DIR, "single_image_dailylife/single_image")

ALL_COMB_DAILYLIFE_FILES = os.listdir(ALL_COMB_DAILYLIFE_DIR)
ALL_COMB_DAILYLIFE_FILES = [f for f in sorted(ALL_COMB_DAILYLIFE_FILES) if f.endswith(".png")]
ALL_SEQ_DAILYLIFE_FOLDERS = os.listdir(ALL_SEQ_DAILYLIFE_DIR)
ALL_SEQ_DAILYLIFE_FOLDERS = sorted(ALL_SEQ_DAILYLIFE_FOLDERS)

# [COMBINED DATASET]
# sampled: we want to reduce the size for this project, which is using only top 100 sequences available on the dataset
# incorrect: the pool to pick image/sequence as incorrect pertubation/noise to our sampled dataset. we picked the last 30 sequences for this purpose
COMB_SAMPLED_DATASET = [os.path.join(ALL_COMB_DAILYLIFE_DIR, filename) for filename in ALL_COMB_DAILYLIFE_FILES[:N_SAMPLED]]
COMB_INCORRECT_DATASET = [os.path.join(ALL_COMB_DAILYLIFE_DIR, filename) for filename in ALL_COMB_DAILYLIFE_FILES[-N_INCORRECT:]]

# [SEQUENTIAL DATASET]
# sampled: we want to reduce the size for this project, which is using only top 100 sequences available on the dataset
# incorrect: the pool to pick image/sequence as incorrect pertubation/noise to our sampled dataset. we picked the last 30 sequences for this purpose
SEQ_SAMPLED_DATASET = [os.path.join(ALL_SEQ_DAILYLIFE_DIR, filename) for filename in ALL_SEQ_DAILYLIFE_FOLDERS[:N_SAMPLED]]
SEQ_INCORRECT_DATASET = [os.path.join(ALL_SEQ_DAILYLIFE_DIR, filename) for filename in ALL_SEQ_DAILYLIFE_FOLDERS[-N_INCORRECT:]]

In [ ]:
# make sure the combined & sequence dataset is consistent

def check_consistency(c_dataset, s_dataset):
  try:
    for i, path in enumerate(c_dataset):
      combined_filename = path.split("/")[-1].replace(".png", "")
      sequential_folder = s_dataset[i].split("/")[-1]
      if  combined_filename != sequential_folder:
        raise Exception
  except Exception:
    print(f"[ERROR INCONSISTENCY FOUND] combined_filename: {combined_filename}, sequential_folder: {sequential_folder}")

check_consistency(COMB_SAMPLED_DATASET, SEQ_SAMPLED_DATASET)
check_consistency(COMB_INCORRECT_DATASET, SEQ_INCORRECT_DATASET)

## Generate Incorrect Information

The incorrect information will be generated by MMLM using the combined input then later manually fixed in order to make sure the generated hallucinated information and ground truth contradict the image content. To create a high-quality list of hallucinations, we use Gemini and GPT-4 as MMLMs, allowing us to have an option between the outputs. <br>

The hallucinated informations we generated consist of:
- hallucinated object keyword list (generated by task 1)
- hallucinated action/behaviour keyword list (generated by task 2)
- generated description containing both hallucinated objects & actions (generated by task 3)

Note that the task 1 actually is inspired from the Mementos paper particularly by the evaluation approach in [here](https://github.com/umd-huang-lab/Mementos/blob/main/GPT-4-assisted_evaluation.ipynb). The idea remains the same, but since the Mementos did not provide with the extracted keywords, we ran the exact prompt used in the code mentioned above to generate them. This allows us to use the keywords for the next tasks.

Based on the prompt from task 1, we adapted the prompts so that we can use it for task 2 & 3

References:
- [create google api key](https://aistudio.google.com/app/u/6/apikey)
- [example notebook from Gemini](https://colab.research.google.com/github/google-gemini/cookbook/blob/main/templates/aistudio_gemini_prompt_freeform.ipynb#templateParams=%7B%22model%22%3A%22gemini-1.5-flash%22%2C%22generation_config_b64%22%3A%22eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9%22%2C%22user_input_b64%22%3A%22SU5TRVJUX0lOUFVUX0hFUkU%3D%22%2C%22contents_b64%22%3A%22W10%3D%22%7D&importGeminiApiKey=true&sandboxMode=true&scrollTo=yoL3p3KPylFW) (not a good guide)
- [Gemini API Documentation](https://ai.google.dev/api/generate-content?authuser=6#image)

In [ ]:
import pandas as pd
from string import Template
from google.colab import userdata
import ast
import time
from tqdm import tqdm

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"
!pip install -U -q "openai==0.28"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai

GEMINI_MODEL_NAME = "gemini-1.5-flash"
gemini_api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=gemini_api_key)

model = genai.GenerativeModel(GEMINI_MODEL_NAME)

In [ ]:
import openai

openai.api_key = userdata.get('OPENAI_API_KEY')

# Define a reusable function
def generate_chat_response(message, temperature=0, max_tokens=1000):
    messages = [
        {"role": "user", "content": []}
    ]

    messages[0]["content"].append({
        "type": "text",
        "text": message})

    return openai.ChatCompletion.create(
        model="gpt-4-1106-preview",  # Set the default model here
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )

`dl_description.csv` serves as a file that contains ground truth

In [ ]:
DESC_PATH = "/content/Mementos/dl_description.csv"
desc_df = pd.read_csv(DESC_PATH,  encoding='ISO-8859-1')

In [ ]:
filename_list = [filename.split("/")[-1] for filename in COMB_SAMPLED_DATASET]
filtered_desc_df = desc_df[desc_df["image_name"].isin(filename_list)]

In [ ]:
filtered_desc_df.shape

(100, 2)

In [ ]:
filtered_desc_df.head()

,image_name,description
0,rw_10011555465.png,The image captures a sequence of movements fro...
1,rw_10111267264.png,The image shows five children standing in a li...
2,rw_10186675055.png,The series of images captures a young girl in ...
3,rw_10192494165.png,"In the image, two children are seated at the b..."
4,rw_10289713176.png,"In the series of images, we see a man in black..."


In [ ]:
# later we will export this as csv
result = {
    "object_list": [],
    "action_list": [],
    "hallucinated_object_list": [],
    "hallucinated_action_list": [],
    "incorrect_description": []
}

In [ ]:
# INCORRECT_DESC_PATH = "" # replace it with your path
INCORRECT_DESC_PATH = "/content/drive/MyDrive/Colab Notebooks/Grounded Language Processing [146078]/Projects/incorrect_description.csv"

In [ ]:
kw_extraction_template = Template("Please extract the object and action words or phrases from the following text. The objects should have a tangible meaning and consist of no more than two words; non-tangible objects should not be extracted. The action words or phrases should only relate to the extracted objects. Also, you must convert the corresponding actions to their complete root form. Please directly output the final object and action lists.\nHere is an example:\n The sequence of images captures a dog's cautious interaction with a metal toy inside a house. The dog appears wary and maintains a distance from the unfamiliar object, barking to express its disapproval and possibly intimidation. As the toy moves, the dog's reaction is to bark and lean backward, showing a clear sign of being unsettled by the toy's motion. When the toy momentarily ceases movement, the dog also stops, remaining alert and attentive. At the end of the image, when the toy comes to a halt, the dog looks up, still processing the strange encounter with the inanimate object.\nThe lists are\nObject list: [dog, toy, house]\nAction list: [interaction, bark, express intimidation, move, lean backward, stop, look up]\nHere is the paragraph:\n $gt_desc. \nThe lists are:")
kw_hallucinate_template = Template("The given lists are object list and action list consecutively. Please create a hallucinated list that contains object words that are not related with the given object list and a hallucinated list that contains action words that are not related with the given action list. However, I want the hallucinated object list still corresponds with the hallucinated action list, which means the objects in the hallucinated object list can do the actions in hallucinated action list. Also, you must convert the hallucinated actions to their complete root form.\nThe lists generated must be in list format [item1, item2, item3].\n For example the lists are\nObject list: [dog, toy, house]\nAction list: [interaction, bark, express intimidation, move, lean backward, stop, look up]\nThe hallucinated lists are:\nObject list: [cat, sofa, ball]\nAction list: [play, nap, land, turn left]\nHere are the given lists:\nObject list:\n $object_list\n Action list:\n $action_list. \nThe hallucinated lists are:")
incorrect_desc_template = Template("Please create a paragraph based on given object list and action list. The objects in the object list do the actions in action list. The action in the action list are in the root form. You can convert it into any tense form, the only condition is that the object and action are connected in plausible way.\nFor example the lists are\nObject list: [dog, toy, house]\nAction list: [interaction, bark, express intimidation, move, lean backward, stop, look up]\nThe generated paragraph is:\nThe sequence of images captures a dog's cautious interaction with a metal toy inside a house. The dog appears wary and maintains a distance from the unfamiliar object, barking to express its disapproval and possibly intimidation. As the toy moves, the dog's reaction is to bark and lean backward, showing a clear sign of being unsettled by the toy's motion. When the toy momentarily ceases movement, the dog also stops, remaining alert and attentive. At the end of the image, when the toy comes to a halt, the dog looks up, still processing the strange encounter with the inanimate object.\nHere are the given lists:\nObject list:\n $object_list\n Action list:\n $action_list. \nThe paragraph is:")

## Declare the Regex Function

In [ ]:
import re

# this pattern is used to capture list in a string. for example "[dog, toy, house]"
list_pattern = re.compile(r"\[([^\]]*)\]")

As a precautions, in case that the MMLMs have difficulty in parsing the list, which is why we provided the function to clean the apostrophe in a string of list. This function can be used when/if needed

In [ ]:
"""
since the string can be ['robot arm', 'telescope', 'dancer's legs', 'puppet']
it will be a problem to convert from a string to a list because an apostrophe inside the word (eg. 'dancer's legs')
the 's make the case tricky as `ast` library will think it is the end of the word, but it isn't.

so, we would like to convert the string to ["robot arm", "telescope", "dancer's legs", "puppet"]
to make sure ast library works correctly

what we want to do, replace ' in these locations:
# 1. After a [ or a comma
# 2. Before a ]
"""
uniquote_pattern = re.compile(r"\[(')|(')\,\s?(')|(')\]")

In [ ]:
# Sample text with single quotes inside list items
text = "**Hallucinated Object List:** ['robot arm', 'telescope', 'dancer's legs', 'puppet']"

# Function to replace single quotes with double quotes
def replace_groups(match):
    # Replace the single quote in the matched group with double quotes
    if match.group(1) == "'":
      return '["'
    if match.group(2) == "'" and match.group(3) == "'":
      return '", "'
    if match.group(4) == "'":
      return '"]'
    else:
      return match.group(0)

# Replace single quotes with double quotes
formatted_text = uniquote_pattern.sub(replace_groups, text)

print(formatted_text)

# h_object_list = uniquote_pattern.sub(replace_groups, h_object_list)
# h_object_list = ast.literal_eval(h_object_list)
# h_action_list = uniquote_pattern.sub(replace_groups, h_action_list)
# h_action_list = ast.literal_eval(h_action_list)

**Hallucinated Object List:** ["robot arm", "telescope", "dancer's legs", "puppet"]


## Task 1: Keyword Extraction (inspired by the Mementos paper)

In [ ]:
for desc in tqdm(filtered_desc_df["description"].tolist(), "keyword extraction"):
  # gemini
  # response = model.generate_content([kw_extraction_template.substitute(gt_desc=desc)])
  # response = response.text.split("\n")

  # chat-gpt
  object_list, action_list = [], []
  while not object_list or not action_list:
    try:
      response = generate_chat_response(message=kw_extraction_template.substitute(gt_desc=desc))
      response = response.choices[0]["message"]["content"].split("\n")

      for line in response:
        if "object list" in line.lower():
          m = list_pattern.search(line)
          object_list = m.group(0) if m else []
          # object_list = ast.literal_eval(object_list)

        if "action list" in line.lower():
          m = list_pattern.search(line)
          action_list = m.group(0) if m else []
          # action_list = ast.literal_eval(action_list)

      if not object_list or not action_list:
        raise Exception
    except:
      print("FAILED TO EXTRACT OBJECT & ACTION LIST")
      print(response)
      time.sleep(5)

  result["object_list"].append(object_list)
  result["action_list"].append(action_list)
  time.sleep(10)

keyword extraction: 100%|██████████| 100/100 [08:51<00:00,  5.32s/it]


In [ ]:
len(result["object_list"])

100

In [ ]:
## trial for using different temperature in gemini

generationConfig = {
    "temperature": 0.7,
}

model.generate_content([kw_hallucinate_template.substitute(
                                                        object_list=object_list, \
                                                        action_list=action_list)],
                                    generation_config=generationConfig).text

'Object list: [bird, nest, worm]\nAction list: [fly, build, eat, turn]\n'

In [ ]:
## trial for using different temperature in chatgpt
response = generate_chat_response(message=kw_hallucinate_template.substitute(
                                                        object_list=result["object_list"][11], \
                                                        action_list=result["action_list"][11]),
                                  temperature=0.7)
response.choices[0]["message"]["content"].split("\n")

['Given the constraints that the hallucinated object list must contain items unrelated to the original object list but able to perform the actions in the hallucinated action list, and that the hallucinated action list contains actions unrelated to the original action list but in their root forms, here are the hallucinated lists:',
 '',
 'Hallucinated Object List:',
 '[fish, airplane, curtain, robot, tree, cyclist, bird, kite, snake, dancer]',
 '',
 'Hallucinated Action List:',
 '[swim, fly, flutter, compute, grow, pedal, chirp, soar, slither, dance]']

## Task 2: Hallucinated Keyword Generation

In [ ]:
result["hallucinated_object_list"] = []
result["hallucinated_action_list"] = []

In [ ]:
for object_list, action_list in tqdm(zip(result["object_list"], result["action_list"]), "keyword hallucination generation"):
  # gemini
  # response = model.generate_content([kw_hallucinate_template.substitute(
  #                                                       object_list=object_list, \
  #                                                       action_list=action_list)])
  # response = response.text.split("\n")

  # h_object_list, h_action_list = [], []
  # for line_idx, line in enumerate(response):
  #   if "object list" in line.lower():
  #     m = list_pattern.search(line)
  #     h_object_list = m.group(0) if m else []
  #   if "action list" in line.lower():
  #     m = list_pattern.search(line)
  #     h_action_list = m.group(0) if m else []

    # for chatgpt
  h_object_list, h_action_list = [], []
  while not h_object_list or not h_action_list:
    try:
      response = generate_chat_response(message=kw_hallucinate_template.substitute(
                                                              object_list=object_list, \
                                                              action_list=action_list),
                                        temperature=0.7)
      response = response.choices[0]["message"]["content"].split("\n")
      for line_idx, line in enumerate(response):
        if "hallucinated object list" in line.lower():
          m = list_pattern.search(f"{line} {response[line_idx+1]}" if line_idx + 1 < len(response) else f"{line}")
          h_object_list = m.group(0) if m else []

        if "hallucinated action list" in line.lower():
          m = list_pattern.search(f"{line} {response[line_idx+1]}" if line_idx + 1 < len(response) else f"{line}")
          h_action_list = m.group(0) if m else []

      if not h_object_list or not h_action_list:
        raise Exception
    except:
      print("FAILED TO HALLUCINATED EXTRACT OBJECT & ACTION LIST")
      print(response)
      time.sleep(5)

  if not h_object_list or not h_action_list:
    print("failed to create hallucination lists")
    raise Exception

  result["hallucinated_object_list"].append(h_object_list)
  result["hallucinated_action_list"].append(h_action_list)
  time.sleep(10)

keyword hallucination generation: 14it [02:44, 12.18s/it]

FAILED TO HALLUCINATED EXTRACT OBJECT & ACTION LIST
['Hallucinated Object List:', '[bird, kite, boat, robot, garden]', '', 'Hallucinated Action List:', '[fly, glide, sail, compute, grow]', '', 'The hallucinated object list consists of items that are not related to girls, roller toys, walls, men, or areas. Similarly, the hallucinated action list consists of actions not related to pushing, holding onto, controlling, hitting, approaching, assisting, shifting, regaining attention, or touching. However, the hallucinated objects can perform the hallucinated actions (e.g., a bird can fly, a kite can glide, a boat can sail, a robot can compute, and a garden can grow).']


keyword hallucination generation: 46it [09:06, 11.39s/it]

FAILED TO HALLUCINATED EXTRACT OBJECT & ACTION LIST
['For the hallucinated lists, we need to choose objects and actions that are not present in the provided lists, while making sure that the objects can perform the actions in the hallucinated action list. ', '', 'Hallucinated Object List:', '[robot, artist, bird, computer, kite, bicycle, gardener]', '', 'Hallucinated Action List:', '[calculate, paint, fly, process, glide, pedal, plant]', '', 'These lists ensure that each object can perform the actions in the corresponding hallucinated action list, while avoiding overlap with the given lists.']


keyword hallucination generation: 57it [11:18, 11.29s/it]

...

keyword hallucination generation: 85it [17:07, 12.28s/it]

FAILED TO HALLUCINATED EXTRACT OBJECT & ACTION LIST
["Certainly! Keeping in mind that the hallucinated object list should be unrelated to the given object list but still correspond to the hallucinated action list, here's what I've come up with:", '', 'Hallucinated Object List:', '[birds, kites, robots, dancers]', '', 'Hallucinated Action List:', '[fly, glide, compute, dance]', '', 'Each object on the hallucinated list can perform the actions described in the hallucinated action list. Birds can fly, kites can glide, robots can compute, and dancers can dance.']


keyword hallucination generation: 99it [19:57, 11.96s/it]

FAILED TO HALLUCINATED EXTRACT OBJECT & ACTION LIST
['Here are the hallucinated lists that follow your instructions:', '', 'Hallucinated Object List:', '[fish, kite, computer, bird]', '', 'Hallucinated Action List:', '[swim, fly, compute, sing]', '', 'In this scenario, the objects in the hallucinated object list can perform the actions in the hallucinated action list. Fish can swim, kites can fly, computers can compute (process data), and birds can sing. These actions are converted to their base or root forms as requested.']


keyword hallucination generation: 100it [20:18, 12.18s/it]


## Task 3: Incorrect Description Generation

In [ ]:
for object_list, action_list in tqdm(zip(result["hallucinated_object_list"], result["hallucinated_action_list"]), "generate incorrect description"):
  # gemini
  # response = model.generate_content([incorrect_desc_template.substitute(
  #                                                       object_list=object_list, \
  #                                                       action_list=action_list)])
  # result["incorrect_description"].append(response.text)

  # chat-gpt
  response = generate_chat_response(message=incorrect_desc_template.substitute(
                                                          object_list=object_list, \
                                                          action_list=action_list),
                                    temperature=0.7)
  result["incorrect_description"].append(response.choices[0]["message"]["content"].replace("\n", " "))

  time.sleep(10)

generate incorrect prompt: 100it [24:50, 14.91s/it]


## Export it as CSV

In [ ]:
filtered_desc_df["object_list"] = result["object_list"]
filtered_desc_df["action_list"] = result["action_list"]
filtered_desc_df["hallucinated_object_list"] = result["hallucinated_object_list"]
filtered_desc_df["hallucinated_action_list"] = result["hallucinated_action_list"]
filtered_desc_df["incorrect_description"] = result["incorrect_description"]
# filtered_desc_df.to_csv(INCORRECT_DESC_PATH, index=False)

<ipython-input-154-41cb27707ea8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_desc_df["object_list"] = result["object_list"]
<ipython-input-154-41cb27707ea8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_desc_df["action_list"] = result["action_list"]
<ipython-input-154-41cb27707ea8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [ ]:
INCORRECT_DESC_PATH

'/content/drive/MyDrive/Colab Notebooks/Grounded Language Processing [146078]/Projects/incorrect_description.csv'